In [18]:
import os
import random
import pickle
from typing import Tuple, Union
import warnings
warnings.simplefilter("ignore", UserWarning)
from tqdm import tqdm
import numpy as np
import torch
import torch.multiprocessing as mp
from environment import Environment
from model import Network
import config
import copy
import ollama
from openai import OpenAI

os.envirion["OPENAI_API_KEY"] = ""
client = OpenAI()

In [ ]:
# negotiator를 구성하기 위한 union find 알고리즘
def find(parent, i):
    if parent[i] == i:
        return i
    else:
        return find(parent, parent[i])

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)

    if xroot != yroot:
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

def merge_sets(lists):
    element_to_index = {}
    for i, subset in enumerate(lists):
        for element in subset:
            element_to_index[element] = i

    parent = [i for i in range(len(lists))]
    rank = [0] * len(lists)

    for subset in lists:
        first_element = subset[0]
        for element in subset[1:]:
            union(parent, rank, find(parent, element_to_index[first_element]), find(parent, element_to_index[element]))

    new_sets = {}
    for element in element_to_index:
        root = find(parent, element_to_index[element])
        if root not in new_sets:
            new_sets[root] = set()
        new_sets[root].add(element)

    return [list(s) for s in new_sets.values()]

In [ ]:
# 방향 정의
directiondict = {
    'stay': 4, 'north': 0, 'south': 1, 'west': 2, 'east': 3
}

# 프롬프트
class gpt4pathfinding:
    def detection(self, agents_state):
        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 MAPF 문제에서 에이전트들의 데드락/라이브락 여부 감지를 위해 호출된 관리자입니다. 당신은 에이전트들의 움직임을 통해 에이전트들이 각각 어떤 상태에 있는지 추론할 수 있는 능력을 가지고 있습니다."},
            {"role": "user", "content":
                f"""
                당신은 에이전트들의 32번간의 action log를 통해 에이전트들의 데드락/라이브락 여부를 확인해야 합니다.
                
                데드락/라이브락으로 분류되는 상태들은 다음과 같습니다:
                not arrived인 상태로 이동하지 않는 상태
                not arrived인 상태로 32번의 행동을 보았을 때 의미있는 좌표의 변화를 나타내지 못하고 배회하는 상태

                데드락/라이브락으로 분류되지 않는 상태들은 다음과 같습니다:
                not arrived였지만 특정 시점에 arrived가 되어 계속 멈춰있는 상태
                계속 not arrived이지만 일관성 있는 좌표의 변화를 나타내며 이동하는 상태

                {agent_state}

                현재 데드락이 존재하는지 yes와 no로 나타내고,
                각각의 에이전트가 데드락/라이브락 상태에 있는지 혹은 정상적인 상태에 있는지 yes or no로 나타내시오.
                """
            }],
        )
        return response.choices[0].message.content
    
    def give_way(self, agents_not_arrived, gridmap):
        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 MAPF 문제에서 잠재적인 충돌을 막기 위해 호출된 관리자입니다. 당신은 그를 위한 경로 계획 능력과 추론 능력을 갖추고 있습니다."},
            {"role": "user", "content":
                f"""
                당신들은 super agent의 정해진 경로에서 다른 에이전트들을 이동시켜 잠재적인 충돌 상황을 회피해야 합니다.
                당신이 이동시켜야 할 에이전트의 목록은 다음과 같습니다: {}
                현재 상황의 맵은 다음과 같습니다: {}
                5는 관측이 불가능한 미지의 공간, 1은 장애물, 0은 빈 공간이며, 3은 super agent의 잠재적인 이동 경로, 4는 super agent의 위치입니다.
                각 에이전트는 east, west, north, south, stay의 4개의 옵션을 가지고 있습니다.
                당신은 json 형식으로 순서대로 4번의 에이전트 계획을 세워야 합니다.
                출력 형식: gridmap 상의 각 에이전트들의 4번간의 행동들"""
            }],
        )
        return response.choices[0].message.content
    
pathfinder = gpt4pathfinding()

In [19]:
torch.manual_seed(config.test_seed)
np.random.seed(config.test_seed)
random.seed(config.test_seed)
DEVICE = torch.device('cpu')
torch.set_num_threads(1)

In [20]:
def create_test(test_env_settings: Tuple = config.test_env_settings, num_test_cases: int = config.num_test_cases):
    '''
    create test set
    '''

    for map_length, num_agents, density in test_env_settings:

        name = f'./test_set/{map_length}length_{num_agents}agents_{density}density.pth'
        print(f'-----{map_length}length {num_agents}agents {density}density-----')

        tests = []

        env = Environment(fix_density=density, num_agents=num_agents, map_length=map_length)

        for _ in tqdm(range(num_test_cases)):
            tests.append((np.copy(env.map), np.copy(env.agents_pos), np.copy(env.goals_pos)))
            env.reset(num_agents=num_agents, map_length=map_length)
        print()

        with open(name, 'wb') as f:
            pickle.dump(tests, f)

In [21]:
def code_test():
    env = Environment()
    network = Network()
    network.eval()
    obs, last_act, pos = env.observe()
    network.step(torch.as_tensor(obs.astype(np.float32)).to(DEVICE), 
                                                    torch.as_tensor(last_act.astype(np.float32)).to(DEVICE), 
                                                    torch.as_tensor(pos.astype(int)))

In [22]:
def test_model(model_range: Union[int, tuple], test_set=config.test_env_settings):
    '''
    test model in 'saved_models' folder
    '''
    network = Network()
    network.eval()
    network.to(DEVICE)

    pool = mp.Pool(mp.cpu_count()//2)

    if isinstance(model_range, int):
        state_dict = torch.load(os.path.join(config.save_path, f'{model_range}.pth'), map_location=DEVICE)
        network.load_state_dict(state_dict)
        network.eval()
        network.share_memory()

        
        print(f'----------test model {model_range}----------')

        instance_id = 0

        for case in test_set:
            print(f"test set: {case[0]} env {case[1]} agents")
            with open('./test_set/{}_{}agents.pth'.format(case[0], case[1]), 'rb') as f:
                tests = pickle.load(f)

            test = tests[0]
            ret = test_one_case((test, network, instance_id))

            success, steps, num_comm = ret

            # instance_id_base = instance_id
            # tests = [(test, network, instance_id_base + i) for i, test in enumerate(tests)]
            # ret = pool.map(test_one_case, tests)

            # success, steps, num_comm = zip(*ret)

            # print("success rate: {:.2f}%".format(sum(success)/len(success)*100))
            # print("average step: {}".format(sum(steps)/len(steps)))
            # print("communication times: {}".format(sum(num_comm)/len(num_comm)))
            # print()

            instance_id += len(tests)

    elif isinstance(model_range, tuple):

        for model_name in range(model_range[0], model_range[1]+1, config.save_interval):
            state_dict = torch.load(os.path.join(config.save_path, f'{model_name}.pth'), map_location=DEVICE)
            network.load_state_dict(state_dict)
            network.eval()
            network.share_memory()


            print(f'----------test model {model_name}----------')

            instance_id = 0

            for case in test_set:
                print(f"test set: {case[0]} length {case[1]} agents {case[2]} density")
                with open(f'./test_set/{case[0]}length_{case[1]}agents_{case[2]}density.pth', 'rb') as f:
                    tests = pickle.load(f)

                test = tests[0]
                ret = test_one_case((test, network, instance_id))

                success, steps, num_comm = ret

                # instance_id_base = instance_id
                # tests = [(test, network, instance_id_base + i) for i, test in enumerate(tests)]
                # ret = pool.map(test_one_case, tests)

                # success, steps, num_comm = zip(*ret)

                # print("success rate: {:.2f}%".format(sum(success)/len(success)*100))
                # print("average step: {}".format(sum(steps)/len(steps)))
                # print("communication times: {}".format(sum(num_comm)/len(num_comm)))
                # print()

                instance_id += 1

            print('\n')

In [40]:
def test_one_case(args):

    env_set, network, instance_id = args

    env = Environment()
    env.load(np.array(env_set[0]), np.array(env_set[1]), np.array(env_set[2]))
    obs, last_act, pos = env.observe()
    
    done = False
    network.reset()

    num_agents = len(env_set[1])

    step = 0
    num_comm = 0
    not_deadlock = False

    while not done and env.steps < config.max_episode_length // 2:
        actions, _, _, _, comm_mask = network.step(torch.as_tensor(obs.astype(np.float32)).to(DEVICE), 
                                                    torch.as_tensor(last_act.astype(np.float32)).to(DEVICE), 
                                                    torch.as_tensor(pos.astype(int)))
        (obs, last_act, pos), _, done, _ = env.step(actions)
        env.save_frame(step, instance_id)
        step += 1
        num_comm += np.sum(comm_mask)

    while not done and env.steps < config.max_episode_length:
        if (env.steps - (config.max_episode_length // 2)) % 32 == 0:

            env_copy = copy.deepcopy(env)
            plan = []
            not_arrived = []
            sim_obs, sim_last_act, sim_pos = env_copy.observe()

            for _ in range(32):
                if env_copy.steps >= config.max_episode_length:
                    break
                actions, _, _, _, comm_mask = network.step(torch.as_tensor(sim_obs.astype(np.float32)).to(DEVICE), 
                                                            torch.as_tensor(sim_last_act.astype(np.float32)).to(DEVICE), 
                                                            torch.as_tensor(sim_pos.astype(int)))
                plan.append((actions, comm_mask, copy.deepcopy(sim_pos)))
                (sim_obs, sim_last_act, sim_pos), _, sim_done, _ = env_copy.step(actions)
                for i in range(num_agents):
                    if not np.array_equal(env_copy.agents_pos[i], env_copy.goals_pos[i]):
                        not_arrived.append(i)

            planned_steps_dict = {i: [] for i in not_arrived}
            for i in plan:
                actions, _, positions = i
                for agent_idx in not_arrived:
                    position = positions[agent_idx]
                    # 목표 위치와 현재 위치를 비교하여 도달 여부 판단
                    arrived_status = "arrived" if np.array_equal(position, env.goals_pos[agent_idx]) else "not arrived"
                    planned_steps_dict[agent_idx].append(f"Action: {actions[agent_idx]}, Position: {position}, Status: {arrived_status}")

            # 결과를 n: [], m: [], o: [] 형식으로 출력
            for agent_idx in planned_steps_dict:
                print(f"{agent_idx}: {planned_steps_dict[agent_idx]}")

            # 이 부분에 openai 입출력, 입력은 텍스트, 출력은 전체에서 데드락/라이브락 유무, 그리고 데드락과 라이브락에 해당하는 에이전트 번호
            deadlocked_agents = []

            if len(deadlocked_agents) == 0:
                not_deadlock = True
            
            if not_deadlock:
                for actions, comm_mask, _ in plan:
                    if env.steps >= config.max_episode_length:
                        break
                    (obs, last_act, pos), _, done, _ = env.step(actions)
                    env.save_frame(step, instance_id)
                    step += 1
                    num_comm += np.sum(comm_mask)

                not_deadlock = False
            else:
                agent_super = max(deadlocked_agents, key=lambda i: np.sum(np.abs(env.agents_pos[i] - env.goals_pos[i])))
                # 32번씩 끊으므로 이 작업은 8번까지만 할 수 있음
                for _ in range(8):
                    if np.array_equal(position, env.goals_pos[agent_idx]):
                        break
                    # super를 heuristic guide에 따라 4번 이동
                    manual_actions = [0 for _ in range(num_agents)]
                    directions = []
                    if obs[agent_super][2][4, 4] == 1:
                        directions.append('north')
                    if obs[agent_super][3][4, 4] == 1:
                        directions.append('south')
                    if obs[agent_super][4][4, 4] == 1:
                        directions.append('west')
                    if obs[agent_super][5][4, 4] == 1:
                        directions.append('east')
                    heuristic_direction = directions
                    direction = random.choice(heuristic_direction)
                    manual_actions[agent_super] = directiondict[direction]
                    # 4번 이동하고 나서 좌표들을 기록해야 함
                    # 기록한 좌표를 기반으로 밑의 FOV에 색칠을 해야 함
                    
                    # 마주친 에이전트들은 피해야 함 openai 입출력
                    #각 에이전트들의 시야에 있는 자신과 다른 에이전트들
                    FOV_agents = []
                    for i in range(num_agents):
                        if np.any(env.observe()[0][i][0]):
                            non_zero_elements = env.observe()[0][i][0][env.observe()[0][i][0] != 0].tolist()
                            non_zero_elements = [element - 1 for element in non_zero_elements]
                            non_zero_elements.append(i)
                            FOV_agents.append(non_zero_elements)

                    #알고리즘을 이용해 연결된 집합 찾기
                    connected_sets = merge_sets(FOV_agents)

                    #연결이 있는 모든 에이전트들
                    deadlocked_agents = [item for sublist in connected_sets for item in sublist]
                    manual_actions[near_agent] = openai 출력



                    # 상관없는 에이전트들은 그대로 멈춤
                    for i in 상관없는 집합:
                        for step in range(4):
                            manual_actions_set[step][i] = 멈춤

                    # 4번의 액션을 환경에 직접 실행함
                    for actions in manual_actions_set:
                        (obs, last_act, pos), _, done, _ = env.step(actions)
                        env.save_frame(step, instance_id)
                        step += 1
                        num_comm += np.sum(comm_mask)


                    # 데드락일 경우 32 step과 꼬이는 부분이 있어서 해결해야 함






    return np.array_equal(env.agents_pos, env.goals_pos), step, num_comm

In [41]:
test_model(128000)

C:\Users\Owner\AppData\Local\Temp\ipykernel_11904\1337376553.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(os.path.join(config.save_path, f'{m

----------test model 128000----------
test set: warehouse env 64 agents
24: ['Action: 3, Position: [  4 116], Status: not arrived', 'Action: 3, Position: [  4 117], Status: not arrived', 'Action: 3, Position: [  4 118], Status: not arrived', 'Action: 3, Position: [  4 119], Status: not arrived', 'Action: 3, Position: [  4 120], Status: not arrived', 'Action: 3, Position: [  4 121], Status: not arrived', 'Action: 3, Position: [  4 122], Status: not arrived', 'Action: 3, Position: [  4 123], Status: not arrived', 'Action: 3, Position: [  4 124], Status: not arrived', 'Action: 3, Position: [  4 125], Status: not arrived', 'Action: 3, Position: [  4 126], Status: not arrived', 'Action: 3, Position: [  4 127], Status: not arrived', 'Action: 3, Position: [  4 128], Status: not arrived', 'Action: 3, Position: [  4 129], Status: not arrived', 'Action: 3, Position: [  4 130], Status: not arrived', 'Action: 3, Position: [  4 131], Status: not arrived', 'Action: 3, Position: [  4 132], Status: no